In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "simulation"

## Prerequisites


### Example batch case

The following batch case is used as an example.


In [2]:
from models.case import BatchCase, Transaction

example_case = BatchCase(
    id="swap_usdt_usdc",
    description="Swap 100 USDT for USDC on Uniswap and transfer to Alice",
    steps=[
        Transaction(description="Approve USDT for Uniswap"),
        Transaction(description="Swap 100 USDT to USDC on Uniswap"),
        Transaction(description="Transfer USDC to Alice"),
    ],
)

### Contract addresses

Assume the following contract addresses are known.


In [3]:
# Contract addresses
USDT_ADDR = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
USDC_ADDR = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"
UNISWAP_V2_ROUTER_ADDR = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D"

# Dummy EOA
ALICE_ADDR = "0x436f795B64E23E6cE7792af4923A68AFD3967952"
BOB_ADDR = "0x8c575b178927fF9A70804B8b4F7622F7666bB360"

### Tools

We use web3.py to create the instances of the contracts.


In [4]:
import os
import json
from web3 import Web3

INFURA_API_KEY = os.getenv("INFURA_API_KEY")
web3 = Web3(Web3.HTTPProvider(f"https://mainnet.infura.io/v3/{INFURA_API_KEY}"))

with open("abi/erc20.json") as file:
    erc20_contract_json = json.load(file)

with open("abi/uniswap_v2_router.json") as file:
    uniswap_contract_json = json.load(file)

# Contract instances
usdc_contract = web3.eth.contract(abi=erc20_contract_json)
usdt_contract = web3.eth.contract(abi=erc20_contract_json)
uniswap_contract = web3.eth.contract(abi=uniswap_contract_json)

Example usage of the contract instances.


In [5]:
def get_balance(token_address: str, address: str):
    contract_instance = web3.eth.contract(
        address=token_address, abi=erc20_contract_json
    )
    encoded = contract_instance.encode_abi("balanceOf", args=[address])
    tx = {
        "to": token_address,
        "data": encoded,
    }

    try:
        # Perform the call
        result = web3.eth.call(tx)
        # Decode the result (balance) from bytes to integer
        decoded_result = int.from_bytes(result)
        # Convert to USDC (assuming 6 decimal places)
        usdc_balance = decoded_result / 10**6
        return usdc_balance
    except Exception as e:
        # Handle any errors that occur during the call
        print(f"An error occurred: {e}")


# Get balance of Bob
print("Bob's USDT balance:", get_balance(USDT_ADDR, BOB_ADDR))
print("Bob's USDC balance:", get_balance(USDC_ADDR, BOB_ADDR))

# Get balance of Alice
print("Alice's USDT balance:", get_balance(USDT_ADDR, ALICE_ADDR))
print("Alice's USDC balance:", get_balance(USDC_ADDR, ALICE_ADDR))

Bob's USDT balance: 0.0
Bob's USDC balance: 0.0
Alice's USDT balance: 4.0
Alice's USDC balance: 4.44435


## Simulate the transaction

Refer to the [Tenderly API documentation](https://docs.tenderly.co/node/rpc-reference/ethereum-mainnet/tenderly_simulateBundle) for more details.


In [6]:
from graph.tools.simulation import (
    simulate_transaction,
    TransactionParams,
    TransactionResult,
)

## Example - Successful


In [7]:
import time

example_tx = [
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=USDC_ADDR,
        data=usdc_contract.encode_abi(
            "approve", args=[UNISWAP_V2_ROUTER_ADDR, 1_000_000]
        ),
        value="0x0",
    ),
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=UNISWAP_V2_ROUTER_ADDR,
        data=uniswap_contract.encode_abi(
            "swapExactTokensForTokens",
            args=[
                1_000_000,
                0,
                [USDC_ADDR, USDT_ADDR],
                ALICE_ADDR,
                int(time.time()) + 500,
            ],
        ),
        value="0x0",
    ),
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=USDT_ADDR,
        data=usdc_contract.encode_abi("transfer", args=[BOB_ADDR, 4_000_001]),
        value="0x0",
    ),
]

result = simulate_transaction(example_tx)
result.pretty_print()

#1: Transaction was successful.
-------------------------------------
#2: Transaction was successful.
💰 USDC (0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48): -1,000,000 (0xf4240)
💰 USDT (0xdac17f958d2ee523a2206206994597c13d831ec7): +998,696 (0xf3d28)
-------------------------------------
#3: Transaction was successful.
💰 USDT (0xdac17f958d2ee523a2206206994597c13d831ec7): -4,000,001 (0x3d0901)
-------------------------------------


## Example - Failed


In [8]:
import time

example_tx = [
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=UNISWAP_V2_ROUTER_ADDR,
        data=uniswap_contract.encode_abi(
            "swapExactTokensForTokens",
            args=[
                1_000_000,
                0,
                [USDC_ADDR, USDT_ADDR],
                ALICE_ADDR,
                int(time.time()) + 500,
            ],
        ),
        value="0x0",
    ),
]

result = simulate_transaction(example_tx)
result.pretty_print()

#1: Transaction was failed.
execution reverted: ERC20: transfer amount exceeds allowance
-------------------------------------
